# Task Dispatch - Interactive AgentECS Demo

This notebook demonstrates how AgentECS works through a dynamic task dispatch system.

**What you'll see:**
- How entities, components, and systems work together
- Dynamic agent spawning and cleanup
- Multi-turn LLM reasoning over multiple ticks
- User interaction when agents need clarification

## Prerequisites

```bash
# Install dependencies
pip install agentecs[llm,notebook]

# Set your API key
export ANTHROPIC_API_KEY=your_key_here
```

## Setup

Import modules and configure the environment:

In [1]:
import os

from components import AgentState, Task, TaskQueue, TaskStatus
from systems import (
    agent_cleanup_system,
    agent_processing_system,
    task_assignment_system,
)

from agentecs import World

# Check API key
if not os.getenv("ANTHROPIC_API_KEY"):
    print("⚠️  Warning: ANTHROPIC_API_KEY not set!")
    print("Set it with: os.environ['ANTHROPIC_API_KEY'] = 'your_key_here'")
else:
    print("✅ API key configured")

✅ API key configured


## Define Tasks

These tasks are intentionally designed to trigger agent questions, demonstrating the ASK_USER functionality:

In [2]:
# Tasks that will likely require user input
DEMO_TASKS = [
    "Plan a vacation for me",  # Will ask: where? when? budget?
    "Recommend a book",  # Will ask: genres? fiction/non-fiction?
    "Write a short story",  # Will ask: genre? length? theme?
    "Calculate the tip for a $47.50 restaurant bill",  # Will ask: what percentage?
]

print(f"📋 {len(DEMO_TASKS)} tasks ready:")
for i, task in enumerate(DEMO_TASKS, 1):
    print(f"  {i}. {task}")

📋 4 tasks ready:
  1. Plan a vacation for me
  2. Recommend a book
  3. Write a short story
  4. Calculate the tip for a $47.50 restaurant bill


## Initialize the World

Create the ECS World and register our three systems:

In [3]:
def setup_world(world):
    """Initialize world with tasks and systems."""
    queue = TaskQueue()
    for task_desc in DEMO_TASKS:
        queue.add_task(task_desc)

    world.set_singleton(queue)

    world.register_system(task_assignment_system)
    world.register_system(agent_processing_system)
    world.register_system(agent_cleanup_system)

    print(f"🌍 World initialized with {len(queue.tasks)} unassigned tasks")
    print("🔧 3 systems registered: TaskAssignment, AgentProcessing, AgentCleanup")
    return world


world = setup_world(World())

🌍 World initialized with 4 unassigned tasks
🔧 3 systems registered: TaskAssignment, AgentProcessing, AgentCleanup


## How the Systems Work

The three systems process agents in parallel during each tick:

### 1. Task Assignment System
```
For each unassigned task in queue:
  1. Call LLM to generate specialized system prompt for task
  2. Spawn agent entity with AgentState + Task components
  3. Remove task from queue (now lives on agent entity)
```

### 2. Agent Processing System
```
For each agent with (AgentState, Task):
  If status == IN_PROGRESS:
    1. Build message history from conversation
    2. Add user response if provided
    3. Call LLM with structured output (AgentResponse)
    4. Handle response type:
       - DEEP_THOUGHT: Add to history, keep thinking
       - ASK_USER: Set question, change status to WAITING_FOR_INPUT
       - FINAL_ANSWER: Set result, change status to COMPLETED
    5. Write back updated AgentState and Task
```

### 3. Agent Cleanup System
```
For each agent with (AgentState, Task):
  If task.status == COMPLETED:
    Destroy agent entity (frees resources)
```

**Key insight**: All three systems run **in parallel** on the same snapshot of world state. Changes are buffered and merged at the tick boundary.

## Helper Functions

Functions to inspect world state between ticks:

In [4]:
def show_world_state():
    """Display current state of agents and tasks."""
    queue = world.singleton(TaskQueue)

    print("\n" + "=" * 60)
    print("📊 WORLD STATE")
    print("=" * 60)

    # Unassigned tasks in queue
    if queue and queue.tasks:
        print(f"\n📋 Unassigned tasks in queue: {len(queue.tasks)}")
        for task in queue.tasks[:3]:  # Show first 3
            print(f"   - {task.description[:50]}...")
    else:
        print("\n📋 Queue is empty (all tasks assigned)")

    # Active agents
    agents = list(world.query(AgentState, Task))
    if agents:
        print(f"\n🤖 Active agents: {len(agents)}")
        for entity, agent_state, task in agents:
            status_emoji = {
                TaskStatus.IN_PROGRESS: "⚙️",
                TaskStatus.WAITING_FOR_INPUT: "❓",
                TaskStatus.COMPLETED: "✅",
            }.get(task.status, "❔")

            print(f"   Agent {entity.index}: {status_emoji} {task.status.value}")
            print(f"      Task: {task.description[:40]}...")
            print(f"      Iterations: {agent_state.iteration_count}")

            if task.status == TaskStatus.WAITING_FOR_INPUT:
                print(f"      ❓ Question: {task.user_query}")
            elif task.status == TaskStatus.COMPLETED:
                print(f"      ✅ Result: {task.result[:50]}...")
    else:
        print("\n🤖 No active agents")

    print("=" * 60 + "\n")


def get_waiting_agents():
    """Get agents waiting for user input."""
    waiting = []
    for entity, agent_state, task in world.query(AgentState, Task):
        if task.status == TaskStatus.WAITING_FOR_INPUT:
            waiting.append((entity, agent_state, task))
    return waiting


print("✅ Helper functions defined")

✅ Helper functions defined


## Tick 1: Task Assignment

The first tick will:
1. **TaskAssignmentSystem** finds unassigned tasks in the queue
2. For each task, generates a specialized system prompt via LLM
3. Spawns an agent entity with `AgentState` and `Task` components
4. Removes task from queue (it now lives on the agent entity!)

This demonstrates **dynamic entity spawning** in ECS.

In [5]:
print("🎬 TICK 1: Running task assignment...\n")

# Run one tick
await world.tick_async()

# Show what happened
show_world_state()

🎬 TICK 1: Running task assignment...

Built execution plan with 1 groups 
(0 dev systems isolated, 3 normal systems grouped)
Dev Systems: []
Normal Systems: ['task_assignment_system', 'agent_processing_system', 'agent_cleanup_system']
✨ Spawned agent for task: b7f59a6e... (Plan a vacation for me...)
✨ Spawned agent for task: 3a7bb21a... (Recommend a book...)
✨ Spawned agent for task: 22f882a3... (Write a short story...)
✨ Spawned agent for task: 6022a80c... (Calculate the tip for a $47.50 restaurant bill...)

📊 WORLD STATE

📋 Queue is empty (all tasks assigned)

🤖 Active agents: 4
   Agent 1000: ⚙️ in_progress
      Task: Plan a vacation for me...
      Iterations: 0
   Agent 1001: ⚙️ in_progress
      Task: Recommend a book...
      Iterations: 0
   Agent 1002: ⚙️ in_progress
      Task: Write a short story...
      Iterations: 0
   Agent 1003: ⚙️ in_progress
      Task: Calculate the tip for a $47.50 restauran...
      Iterations: 0



## Tick 2: Initial Agent Processing

The second tick will:
1. **AgentProcessingSystem** queries all entities with `(AgentState, Task)` components
2. For each agent in IN_PROGRESS status, calls Claude with:
   - Agent's specialized system prompt
   - Conversation history (empty on first call)
   - Structured output model: `AgentResponse(reasoning, response_type, message)`
3. Handles responses:
   - **DEEP_THOUGHT**: Adds to conversation history, continues thinking
   - **ASK_USER**: Sets question, waits for input
   - **FINAL_ANSWER**: Marks task complete

This demonstrates **multi-turn reasoning** and **structured LLM output**.

In [6]:
print("🎬 TICK 2: Running agent processing...\n")

await world.tick_async()

show_world_state()

🎬 TICK 2: Running agent processing...

❓ Agent 1000 asks: I'd be happy to help plan your vacation! To create the best itinerary for you, could you share: 1) Where would you like to go (or what type of experience interests you - beach, city, adventure, etc.)? 2) When are you traveling and for how long? 3) What's your budget range? 4) How many people are traveling, and are there any special interests or requirements I should know about?
❓ Agent 1001 asks: I'd love to recommend a book for you! To find the perfect match, could you share: What genres do you typically enjoy? Are there any recent books you've loved? What mood or themes are you interested in exploring right now?
✅ Agent 1002 completed: **The Last Lighthouse Keeper**

The fog rolled in thick that...
✅ Agent 1003 completed: Here are the tip calculations for your $47.50 restaurant bil...

📊 WORLD STATE

📋 Queue is empty (all tasks assigned)

🤖 Active agents: 4
   Agent 1000: ❓ waiting_for_input
      Task: Plan a vacation for me.

## User Interaction: Answer Agent Questions

If any agents are waiting for input, provide answers here.

This demonstrates **external interaction** with the ECS - we query the world state and modify components directly.

In [7]:
waiting = get_waiting_agents()

if waiting:
    print(f"\n❓ {len(waiting)} agent(s) waiting for input:\n")

    for entity, _agent_state, task in waiting:
        print(f"\n🤖 Agent {entity.index} asks:")
        print(f"   {task.user_query}")
        print(f"   Task: {task.description}")

        # Get user response
        response = input("\nYour answer: ")

        # Inject response back into task - must write back!
        task.user_response = response
        task.status = TaskStatus.IN_PROGRESS
        world.set(entity, task)

        print(f"✅ Response recorded for Agent {entity.index}\n")
else:
    print("\n✅ No agents waiting for input")


❓ 2 agent(s) waiting for input:


🤖 Agent 1000 asks:
   I'd be happy to help plan your vacation! To create the best itinerary for you, could you share: 1) Where would you like to go (or what type of experience interests you - beach, city, adventure, etc.)? 2) When are you traveling and for how long? 3) What's your budget range? 4) How many people are traveling, and are there any special interests or requirements I should know about?
   Task: Plan a vacation for me



Your answer:  Instanbul in june


✅ Response recorded for Agent 1000


🤖 Agent 1001 asks:
   I'd love to recommend a book for you! To find the perfect match, could you share: What genres do you typically enjoy? Are there any recent books you've loved? What mood or themes are you interested in exploring right now?
   Task: Recommend a book



Your answer:  i like neil stephensOn


✅ Response recorded for Agent 1001



## Continue Processing: Run More Ticks

Run this cell multiple times to advance the simulation.

Each tick will:
- Process agents that have user responses
- Continue DEEP_THOUGHT chains
- Ask more questions if needed
- Complete tasks
- Clean up finished agents

This demonstrates the **iterative tick-based execution model** of ECS.

In [8]:
# Run one more tick
await world.tick_async()

show_world_state()

# Check for more waiting agents
waiting = get_waiting_agents()
if waiting:
    print(f"\n❗ {len(waiting)} agent(s) need your input - run the interaction cell above!")
else:
    active = sum(1 for _ in world.query(AgentState))
    if active == 0:
        print("\n🎉 All tasks completed!")
    else:
        print(f"\n⏳ {active} agent(s) still thinking...")

Cleaning up agent 1002 (task 22f882a3... completed)
Cleaning up agent 1003 (task 6022a80c... completed)
❓ Agent 1000 asks: Great choice! Istanbul in June is wonderful with warm weather. To plan your trip properly, I need a few more details: How many days will you be staying? What's your approximate budget (total or per day)? How many people are traveling? And are there specific experiences you're interested in (history, food, shopping, nightlife, etc.)?
✅ Agent 1001 completed: Great choice! Neal Stephenson fans appreciate dense, intelle...

📊 WORLD STATE

📋 Queue is empty (all tasks assigned)

🤖 Active agents: 2
   Agent 1000: ❓ waiting_for_input
      Task: Plan a vacation for me...
      Iterations: 2
      ❓ Question: Great choice! Istanbul in June is wonderful with warm weather. To plan your trip properly, I need a few more details: How many days will you be staying? What's your approximate budget (total or per day)? How many people are traveling? And are there specific experiences

## Full Auto-Run Loop

This cell runs ticks automatically until all tasks complete or we hit the limit.

It will pause and ask for user input when agents have questions.

In [5]:
import asyncio

MAX_TICKS = 20
completed_tasks = []

world = setup_world(World())

for tick_num in range(MAX_TICKS):
    print(f"\n{'=' * 60}")
    print(f"⏱️  TICK {tick_num + 1}")
    print("=" * 60)

    # Save completed tasks before cleanup
    for _entity, _agent_state, task in world.query(AgentState, Task):
        if task.status == TaskStatus.COMPLETED and task not in completed_tasks:
            completed_tasks.append(task)

    # Run tick
    await world.tick_async()

    # Handle user queries
    waiting = get_waiting_agents()
    if waiting:
        print(f"\n❓ {len(waiting)} agent(s) need input:\n")

        for entity, _agent_state, task in waiting:
            print(f"🤖 Agent {entity.index}: {task.user_query}")
            print(f"   (Task: {task.description})")
            response = input("   Your answer: ")

            # Must write back mutations!
            task.user_response = response
            task.status = TaskStatus.IN_PROGRESS
            world.set(entity, task)
            print()

    # Check if done
    active_agents = sum(1 for _ in world.query(AgentState))
    if active_agents == 0:
        print(f"\n✅ All tasks completed after {tick_num + 1} ticks!")
        break

    await asyncio.sleep(0.5)  # Brief pause for readability

# Final summary
print("\n" + "=" * 60)
print("📊 FINAL RESULTS")
print("=" * 60)

for i, task in enumerate(completed_tasks, 1):
    print(f"\n{i}. {task.description}")
    print(f"   ✅ {task.result}")

print("\n" + "=" * 60)

🌍 World initialized with 4 unassigned tasks
🔧 3 systems registered: TaskAssignment, AgentProcessing, AgentCleanup

⏱️  TICK 1
✨ Spawned agent for task: 5a1da800... (Plan a vacation for me...)
✨ Spawned agent for task: 84c9d43f... (Recommend a book...)
✨ Spawned agent for task: 6191d8c6... (Write a short story...)
✨ Spawned agent for task: a41df511... (Calculate the tip for a $47.50 restaurant bill...)

⏱️  TICK 2
❓ Agent 1000 asks: I'd love to help plan your vacation! To create the perfect trip for you, could you share: Where would you like to go (or what type of destination interests you)? When and for how long? What's your budget range? Who's traveling (solo/couple/family/group)? And what are your main interests (beach relaxation, adventure, culture, food, etc.)?
❓ Agent 1001 asks: I'd love to help you find the perfect book! To give you the best recommendation, could you tell me: What genres do you typically enjoy? Are there any recent books you've loved? What kind of reading mood ar

   Your answer:  instanbul in summer



🤖 Agent 1001: I'd love to help you find the perfect book! To give you the best recommendation, could you tell me: What genres do you typically enjoy? Are there any recent books you've loved? What kind of reading mood are you in right now (something light and fun, thought-provoking, emotional, adventurous)?
   (Task: Recommend a book)


   Your answer:  i have read all Of neil stephensOn




⏱️  TICK 3
Cleaning up agent 1002 (task 6191d8c6... completed)
Cleaning up agent 1003 (task a41df511... completed)
❓ Agent 1000 asks: Great choice! Istanbul is wonderful in summer. To create your perfect itinerary, I need a few more details: How many days will you be visiting? What's your total budget? Who's traveling with you? And what interests you most - historical sites, food tours, shopping, nightlife, or a mix?
✅ Agent 1001 completed: Excellent! Since you've devoured all of Neal Stephenson's wo...

❓ 1 agent(s) need input:

🤖 Agent 1000: Great choice! Istanbul is wonderful in summer. To create your perfect itinerary, I need a few more details: How many days will you be visiting? What's your total budget? Who's traveling with you? And what interests you most - historical sites, food tours, shopping, nightlife, or a mix?
   (Task: Plan a vacation for me)


   Your answer:  just surprise me




⏱️  TICK 4
Cleaning up agent 1001 (task 84c9d43f... completed)
✅ Agent 1000 completed: 🌟 **YOUR SURPRISE ISTANBUL SUMMER ADVENTURE** 🌟

**Duration:...

⏱️  TICK 5
Cleaning up agent 1000 (task 5a1da800... completed)

✅ All tasks completed after 5 ticks!

📊 FINAL RESULTS

1. Write a short story
   ✅ **The Last Customer**

The bell above the door chimed at 11:47 PM, thirteen minutes before closing. Marcus looked up from wiping down the espresso machine, already calculating how much longer this would take. The customer was a woman in her sixties, wearing a rain-soaked coat that dripped onto the café's worn floorboards.

"Are you still open?" she asked, breathless.

Marcus gestured to the chairs already stacked on tables. "Technically."

"I just need coffee. Black. Please." Something in her voice—a tremor, a desperate edge—made him pause mid-excuse.

"Sure. Have a seat."

She chose the only unstacked table, the wobbly one by the window that Marcus kept meaning to fix. While the machine hi

## Key ECS Concepts Demonstrated

### 1. Entities are Just IDs
- Agents are lightweight `EntityId` objects
- No behavior, just identity

### 2. Components are Pure Data
- `Task`: description, status, result, user interaction state
- `AgentState`: system prompt, conversation history, iteration count
- `TaskQueue`: list of unassigned tasks (singleton)

### 3. Systems Define Behavior
- Query entities with specific component combinations
- Read components, compute, write back
- Example: `world(AgentState, Task)` gets agents with tasks

### 4. Task Component Lifecycle
```
UNASSIGNED: Lives in TaskQueue singleton
     ↓
IN_PROGRESS: Moves to agent entity (spawned with AgentState), agent working
     ↓
WAITING_FOR_INPUT: Agent has question (pauses for user)
     ↓
COMPLETED: Task done
     ↓
Agent destroyed, components removed
```

### 5. No Cross-Referencing Needed
- Task and AgentState paired on same entity
- Query returns both: `world(AgentState, Task)`
- Clean, natural ECS pattern

### 6. Dynamic Entity Management
- Agents spawned on-demand (TaskAssignmentSystem)
- Agents destroyed when done (AgentCleanupSystem)
- Components move between entities (Task from queue to agent)

## Experiment: Add New Tasks

Try adding your own tasks and running more ticks!

In [10]:
# Get the queue and add new tasks
queue = world.singleton(TaskQueue)

new_tasks = [
    "Explain quantum computing",
    "Suggest a recipe for dinner",
]

for task_desc in new_tasks:
    queue.add_task(task_desc)
    print(f"➕ Added: {task_desc}")

world.set_singleton(queue)
print(f"\n📋 Queue now has {len(queue.tasks)} unassigned tasks")
print("\n▶️  Run tick cells above to process them!")

➕ Added: Explain quantum computing
➕ Added: Suggest a recipe for dinner

📋 Queue now has 2 unassigned tasks

▶️  Run tick cells above to process them!
